## Imports and data loading

In [36]:
import math
import random
import time

import pandas as pd
import numpy as np

from scipy import sparse
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

from sparselsh import LSH

## Content-based, regression

In [2]:
# load food.com data
directory = 'data/food.com'
df_recipe_rating = pd.read_csv(f'{directory}/recipe_ratings.csv')
df_recipe = pd.read_csv(f'{directory}/recipe.csv')

In [3]:
df_recipe.head()

,recipe_id,name,minutes,n_steps,n_ingredients,calories,fat,sugar,sodium,protein,saturated,carbs
0,137739,arriba baked winter squash mexican style,55,11,7,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,31490,a bit different breakfast pizza,30,9,6,173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,112140,all in the kitchen chili,130,6,13,269.8,22.0,32.0,48.0,39.0,27.0,5.0
3,59389,alouette potatoes,45,11,11,368.1,17.0,10.0,2.0,14.0,8.0,20.0
4,44061,amish tomato ketchup for canning,190,5,8,352.9,1.0,337.0,23.0,3.0,0.0,28.0


In [4]:
df_recipe_rating

,user_id,recipe_id,rating
0,2046,4684,5.0
1,2046,517,5.0
2,1773,7435,5.0
3,1773,278,4.0
4,2046,3431,5.0
...,...,...,...
698896,926904,457971,5.0
698897,2002312797,27208,5.0
698898,1290903,131607,5.0
698899,226867,363072,5.0


In [146]:
#EXERCISE: Build a content-based recommender system that uses linear regression 
#          to predict ratings.
#          Try it out on users with a high number of ratings.
#          Try some train-test split to evaluate performance.

#select the ratings of a specific user 


sorted_users = df_recipe_rating.groupby(["user_id"]).agg(count=("user_id", "count")).sort_values("count", ascending=False)
target_id = sorted_users.index[0]
df_user = df_recipe_rating[df_recipe_rating['user_id']==target_id][['recipe_id','rating']]
df_rec = pd.merge(df_user, df_recipe, on='recipe_id', how='inner')



# define features
features = ["minutes", 
            "n_steps", 
            "n_ingredients", 
            "calories", 
            "fat", 
            "sugar",
            "sodium",
            "protein",
            "saturated",
            "carbs"
        ] 

# split training and test
x = df_rec.drop(["recipe_id" , "name"], axis=1)
y = df_rec["rating"]
x_train, x_test, y_train, y_test = train_test_split(x, y, shuffle=True)


model = LinearRegression();

predicted = model.fit(x_train, y_train).predict(x_test)


expected = []
for index, row in x_test.iterrows():
    expected.append(row['rating'])

mse = (np.square(expected - predicted)).mean()

print(f"mse:  {mse}")

# ...

mse:  1.069372003506123e-31


## Content-based, KNN (with LSH)

In [168]:
directory = 'data/movielens/ml-latest-small'
#directory = 'data/movielens/ml-latest' #change into this one for the full dataset (slow)

df_movies = pd.read_csv(f'{directory}/movies.csv')
df_ratings = pd.read_csv(f'{directory}/ratings.csv')
df_tags = pd.read_csv(f'{directory}/tags.csv')

#transform tags such that they are lower-case, single-word tokens
df_tags['tag'] = df_tags['tag'].apply(lambda x: str(x).lower().replace(' ', '_'))

In [148]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [153]:
df_tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,highly_quotable,1445714996
2,2,60756,will_ferrell,1445714992
3,2,89774,boxing_story,1445715207
4,2,89774,mma,1445715200


### Step1: Calculate item profiles

In [182]:
# calculates the lexicon of most frequent tags.
tag_frequency_threshold = 5 # increase number to filter
# sorted_users = df_recipe_rating.groupby(["user_id"]).agg(count=("user_id", "count")).sort_values("count", ascending=False)
df_lexicon = df_tags.groupby("tag").agg(count=("tag", "count")) # get a dataframe with tags and respective counts
# discard movies with no tags
item_profiles = df_movies[df_movies["movieId"].isin(df_tags["movieId"])]
item_profiles
# you can drop the userId and timestamp columns because we don't care who assigned the tag and when
# ...

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
4,5,Father of the Bride Part II (1995),Comedy
6,7,Sabrina (1995),Comedy|Romance
...,...,...,...
9681,183611,Game Night (2018),Action|Comedy|Crime|Horror
9692,184471,Tomb Raider (2018),Action|Adventure|Fantasy
9709,187593,Deadpool 2 (2018),Action|Comedy|Sci-Fi
9710,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi


In [150]:
#calculate the sparse feature vector based on the TF-IDF of words in documents
#the TF-IDF vectors are saved as sparse representations into the dataframe
df_features = df_tags.groupby('movieId').agg(lambda x: ' '.join(x)).reset_index()
vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split(' ')).fit(sorted(df_features['tag']))
vectorizer.vocabulary_
df_features['feature_vector'] = df_features['tag'].apply(lambda x : vectorizer.transform([x]))
df_features

/var/folders/m5/l9b6fpv12v370f32ct4y00h00000gn/T/ipykernel_64652/2621643708.py:3: FutureWarning: ['userId', 'timestamp'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df_features = df_tags.groupby('movieId').agg(lambda x: ' '.join(x)).reset_index()
/Users/alexanderwermuth/opt/anaconda3/envs/data-in-production/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,movieId,tag,feature_vector
0,1,pixar pixar fun,"(0, 1016)\t0.8992312620852461\n (0, 513)\t0..."
1,2,fantasy magic_board_game robin_williams game,"(0, 1129)\t0.48431677463057304\n (0, 812)\t..."
2,3,moldy old,"(0, 967)\t0.7071067811865475\n (0, 876)\t0...."
3,5,pregnancy remake,"(0, 1106)\t0.6808321878901952\n (0, 1044)\t..."
4,7,remake,"(0, 1106)\t1.0"
...,...,...,...
1567,183611,comedy funny rachel_mcadams,"(0, 1084)\t0.7153847877740599\n (0, 515)\t0..."
1568,184471,adventure alicia_vikander video_game_adaptation,"(0, 1418)\t0.6173008972335793\n (0, 46)\t0...."
1569,187593,josh_brolin ryan_reynolds sarcasm,"(0, 1156)\t0.5453455400986055\n (0, 1147)\t..."
1570,187595,emilia_clarke star_wars,"(0, 1266)\t0.6876494487057803\n (0, 433)\t0..."


### Step2: Index item profiles into LSH

In [209]:
#index all item vectors into LSH
lsh = LSH(4, df_features.shape[1], num_hashtables=1, storage_config={"dict":None})

# print(df_features.shape[1])
# sparse = df_features["feature_vector"].map(lambda a: vectorizer.inverse_transform(a))
csr_matrix(sparse, shape=df_features.shape[1] )
# df_lexicon["count"]
# lsh.index()

#run an example query to the LSH
# lsh.query(...)

AttributeError: 'LSH' object has no attribute 'insert'

### Step 3: Calculate user profile

In [ ]:
# restricts the ratings to the set of most popular movies (optional, not needed for content-based)
numratings_threshold = 0 #increase this number if you want to filter
df_item_popularity = df_ratings[['movieId','rating']].groupby('movieId').count().reset_index()
df_item_popularity.columns = ['movieId','count'] 
df_item_popularity = df_item_popularity.sort_values(by='count', ascending=False)
df_item_popularity = df_item_popularity[df_item_popularity['count'] >= numratings_threshold]
print(f'Number of movies reduced from {len(df_ratings.movieId.unique())} to {len(df_item_popularity.movieId.unique())}')
df_ratings = pd.merge(df_ratings, df_item_popularity, on='movieId', how='inner')[['userId', 'movieId', 'rating']]
df_ratings = df_ratings.sort_values(by='userId')

#rescale the ratings by the user's individual average 
df_ratings['rating_scaled'] = ...

df_ratings.head()

In [ ]:
# join ratings with movie feature vectors
df_profile = pd.merge(df_ratings, df_features[['movieId','feature_vector']],
              on='movieId')
#scaling feature vector by rating (this will take a few minutes)
df_profile['feature_vector_scaled'] = df_profile['rating_scaled'] * df_profile['feature_vector']
df_profile

In [ ]:
start = time.time()
#stack all sparse vectors of user's movies
df_user_vectors = df_profile[['userId', 'feature_vector_scaled']].groupby('userId').agg(sparse.vstack).reset_index()
#compute the average of the vectors without considering the zero entries (this will take a while)
df_user_vectors['feature_vector_scaled'] = df_user_vectors['feature_vector_scaled'].apply(lambda x: csr_matrix(np.nan_to_num(x.sum(axis=0)/x.getnnz(axis=0), 0)))
end = time.time()
print(end - start)
df_user_vectors

### Step 4: Rank potential recommendation candidates

In [ ]:
#pick a target user to provide recommendations to
idx = 42
target_userId = df_user_vectors.iloc[idx].userId

In [ ]:
#get user rating history
df_user_history = ...

#select candidate recommendations to user
df_recommendation = ...

In [ ]:
df_recommendation

In [ ]:
df_user_history.head(10)

In [ ]:
df_user_history.tail(10)

### Step 5: Predict ratings of candidate items

In [ ]:
#index all user vectors into LSH
df_usr = df_profile[df_profile['userId'] == target_userId]
lsh_usr = LSH(...)
lsh_usr.index(..., extra_data=[...]) # repeat for all users. Insert movieid and rating as extra data for future retrieval
lsh_usr

In [ ]:
# compute recommendation
df_recommendation = ...

## Collaborative filtering

In [ ]:
from surprise import SVD
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms.knns import KNNBasic

In [ ]:
directory = 'data/movielens/ml-latest-small'
#directory = 'data/movielens/ml-latest' #change into this one for the full dataset (slow)
df_ratings = pd.read_csv(f'{directory}/ratings.csv')
df_ratings.head()

In [ ]:
# initialize a data reader
reader = Reader(rating_scale=(1, 5))
# provide a dataset with userid, itemtid, and rating in order
data = Dataset.load_from_df(df_ratings[['userId','movieId','rating']], reader)

# surprise has also some built-in datasets that can be imported directly
#data = Dataset.load_builtin('ml-100k')

In [ ]:
# initialize a user-based K nearest neighbors implementation
...
# execute 5-fold cross-validation and measure RMSE and MAE
...